In [14]:
import openke
from openke.config import Trainer, Tester
from openke.module.model import TransE
from openke.module.loss import MarginLoss
from openke.module.strategy import NegativeSampling
from openke.data import TrainDataLoader, TestDataLoader
import torch
import torch.nn.functional as F

In [2]:
# dataloader for training
train_dataloader = TrainDataLoader(
	in_path = "./benchmarks/YAGO3-10/", 
	nbatches = 100,
	threads = 8, 
	sampling_mode = "normal", 
	bern_flag = 1, 
	filter_flag = 1, 
	neg_ent = 25,
	neg_rel = 0)

In [3]:
# dataloader for test
test_dataloader = TestDataLoader("./benchmarks/YAGO3-10/", "link")

In [ ]:
remove_dataloader = TrainDataLoader(
	in_path = "./benchmarks/YAGO3-10/", 
	nbatches = 100,
	threads = 8, 
	sampling_mode = "normal", 
	bern_flag = 1, 
	filter_flag = 1, 
	neg_ent = 25,
	neg_rel = 0)

In [4]:
transe = TransE(
	ent_tot = train_dataloader.get_ent_tot(),
	rel_tot = train_dataloader.get_rel_tot(),
	dim = 200, 
	p_norm = 1, 
	norm_flag = True)
transe.load_checkpoint('./checkpoint/YAGO_TransE.ckpt')

In [19]:
data = next(iter(train_dataloader))
batch_h = torch.tensor(data['batch_h'])
batch_t = torch.tensor(data['batch_t'])
batch_r = torch.tensor(data['batch_r'])
h = transe.ent_embeddings(batch_h)
t = transe.ent_embeddings(batch_t)
r = transe.rel_embeddings(batch_r)
index1 = 0
index2 = 1
embedding1 = torch.cat((h[index1], t[index1], r[index1]))
embedding2 = torch.cat((h[index2], t[index2], r[index2]))
cos = F.cosine_similarity(embedding1.unsqueeze(0), embedding2.unsqueeze(0))

In [20]:
cos

tensor([0.9861], grad_fn=<SumBackward1>)

In [5]:
# define the loss function
model = NegativeSampling(
	model = transe, 
	loss = MarginLoss(margin = 5.0),
	batch_size = train_dataloader.get_batch_size()
)

In [ ]:
# train the model
trainer = Trainer(model = model, data_loader = train_dataloader, train_times = 1000, alpha = 1.0, use_gpu = True)
trainer.run()
transe.save_checkpoint('./checkpoint/YAGO_TransE.ckpt')

Finish initializing...


  0%|                                                                                          | 0/1000 [00:00<?, ?it/s]

Input Files Path : ./benchmarks/YAGO3-10/
The toolkit is importing datasets.
The total of relations is 37.
The total of entities is 123182.
The total of train triples is 1079040.
Input Files Path : ./benchmarks/FB15K237/
The total of test triples is 20466.
The total of valid triples is 17535.
Input Files Path : ./benchmarks/YAGO3-10/
The total of test triples is 5000.
The total of valid triples is 5000.


Epoch 73 | loss: 1.296135:   7%|███▊                                                | 74/1000 [06:01<1:15:20,  4.88s/it]

In [6]:
# test the model
transe.load_checkpoint('./checkpoint/YAGO_TransE.ckpt')
tester = Tester(model = transe, data_loader = test_dataloader, use_gpu = True)
tester.run_link_prediction(type_constrain = False)

  1%|▌                                                                               | 32/5000 [00:00<00:44, 111.81it/s]

Input Files Path : ./benchmarks/YAGO3-10/
The toolkit is importing datasets.
The total of relations is 37.
The total of entities is 123182.
The total of train triples is 1079040.
Input Files Path : ./benchmarks/YAGO3-10/
The total of test triples is 5000.
The total of valid triples is 5000.


100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:18<00:00, 273.82it/s]

0.5443000197410583


(0.3315134644508362,
 1476.792236328125,
 0.5443000197410583,
 0.388700008392334,
 0.21889999508857727)

no type constraint results:
metric:			 MRR 		 MR 		 hit@10 	 hit@3  	 hit@1 
l(raw):			 0.042049 	 4617.669434 	 0.106800 	 0.032800 	 0.008200 
r(raw):			 0.201719 	 462.848999 	 0.587400 	 0.182200 	 0.073000 
averaged(raw):		 0.121884 	 2540.259277 	 0.347100 	 0.107500 	 0.040600 

l(filter):		 0.183610 	 2496.848877 	 0.341000 	 0.202000 	 0.104800 
r(filter):		 0.479416 	 456.735596 	 0.747600 	 0.575400 	 0.333000 
averaged(filter):	 0.331513 	 1476.792236 	 0.544300 	 0.388700 	 0.218900 
0.544300
